<a href="https://colab.research.google.com/github/mgstockwell/water_well_prediction/blob/main/Groundwater_Load_mTPI_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Earth Engine Python API Colab Setup

This notebook connects to the Earth Engine Python API and downloads Earth Engine processed data from images for use by ML algorithms.

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [1]:
# import the Earth Engine API
import os, sys
from google.colab import files
import ee
from google.auth.transport.requests import AuthorizedSession
from google.oauth2 import service_account
import pandas as pd

project = 'msd8654-498-dev'
#!gcloud auth login --project {PROJECT}
if os.path.exists('/content/msd8654-498-dev-1a070409c971.json'):
  print('key exists')
else:
  KEY = files.upload()
service_account = 'my-first-app-92f7826c73ade84fa@msd8654-498-dev.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '/content/msd8654-498-dev-1a070409c971.json')

# Trigger the authentication flow.
# ee.Authenticate()

ee.Initialize(
  credentials=credentials,
  project=project,
  opt_url='https://earthengine-highvolume.googleapis.com'
)

df = pd.DataFrame()
df_state_cds = pd.DataFrame()


Saving msd8654-498-dev-1a070409c971.json to msd8654-498-dev-1a070409c971.json


## Test the API

Test the API by printing the elevation of Mount Everest.

In [2]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


# Connect to BigQuery and Define Functions

In [3]:
import os
import json
import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.colab import auth

google_project_id = 'msd8654-498-dev'
# Set the project id
os.environ.putenv('GOOGLE_CLOUD_PROJECT', google_project_id)

# logon and get credentials.
if os.getenv('GAE_ENV', '').startswith('standard'):
  # Production in the standard environment, all OK
  None
elif os.path.exists(google_project_id + ".json"):
  # Local execution.
  key_path = google_project_id + ".json"
  credentials = service_account.Credentials.from_service_account_file(
      key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
  )
  client = bigquery.Client(credentials=credentials, project=credentials.project_id,)
else:
  auth.authenticate_user()

client = bigquery.Client(project=google_project_id)
print('Authenticated')



Authenticated


## Upload file function

In [4]:
from google.cloud import storage
from google.cloud import bigquery
import urllib.request
import os, datetime

# note: google_project_name  & google_dataset_name at top (global)

def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the google storage bucket."""

    storage_client = storage.Client(project=google_project_id)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to Storage Bucket {} successfully . {}".format(
            source_file_name, destination_blob_name, datetime.datetime.now()
        )
    )

# Import Groundwater base data

Source data is from https://nwis.waterdata.usgs.gov/usa/nwis/gwlevels

Additional datasources and info: https://cloud.google.com/architecture/geospatial-analytics-architecture#coordinate_reference_systems 

## Load Sites data From BQ

The `google.cloud.bigquery` library includes a magic command which runs a query and either displays the result or saves it to a variable as a `DataFrame`.

In [6]:
# Save output in a variable `df`
%%bigquery --project msd8654-498-dev df
SELECT
          DISTINCT site_no,
          station_nm,
          site_tp_cd,
          lat_va,
          long_va,
          dec_lat_va,
          dec_long_va,
          district_cd,
          right(concat('0',state_cd),2) state_cd,
          county_cd,
          well_depth_va,
          well_depth_hue
        FROM
          `msd8654-498-dev.usgs.groundwater_sites_vis`
        WHERE
          well_depth_va IS NOT NULL
          and dec_lat_va is not null
          and dec_long_va is not null
          and cast(state_cd as int) > 29

In [ ]:
df

,site_no,station_nm,site_tp_cd,lat_va,long_va,dec_lat_va,dec_long_va,district_cd,state_cd,county_cd,well_depth_va,well_depth_hue
0,350160106324201,09N.04E.05.323 TJA-3,GW,350200.09,1063243.88,35.033358,-106.545522,35,35,1,521.7,6
1,365204105365501,30N.12E.01.1414 AR-1,GW,365204.09,1053655.07,36.867803,-105.615297,35,35,55,27.0,2
2,350211106315801,09N.04E.05.244 KAFB-0311,GW,350211.18,1063200.46,35.036439,-106.533461,35,35,1,468.0,6
3,350304106340801,10N.03E.36.24 KAFB-16,GW,350305.13,1063409.26,35.051425,-106.569239,35,35,1,1400.0,6
4,350352106334601,10N.04E.30.321 KAFB-3,GW,350354.5,1063346.74,35.065139,-106.562983,35,35,1,900.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...
398658,400120074233501,290888-- Di Obs,GW,400120.97,742333.29,40.022492,-74.392581,34,34,29,20.0,2
398659,400059074223801,290889-- Dk Obs,GW,400100.47,741917.54,40.016797,-74.321539,34,34,29,20.0,2
398660,400156074204601,290904-- Dt Obs,GW,400156.34,742043.83,40.032317,-74.345508,34,34,29,25.0,2
398661,400231074221701,290905-- Dy Obs,GW,400229.24,742204.13,40.041456,-74.367814,34,34,29,27.0,2


## Load State Codes

In [ ]:
# Save output in a variable `df_state_cds`

%%bigquery --project msd8654-498-dev df_state_cds
SELECT
  state_fips_code,
  state_postal_abbreviation,
  state_name,
  state_gnisid
FROM
  `bigquery-public-data.census_utility.fips_codes_states`
  where state_fips_code >'29'


In [ ]:
display(df_state_cds)

,state_fips_code,state_postal_abbreviation,state_name,state_gnisid
0,30,MT,Montana,00767982
1,31,NE,Nebraska,01779792
2,32,NV,Nevada,01779793
3,33,NH,New Hampshire,01779794
4,34,NJ,New Jersey,01779795
5,35,NM,New Mexico,00897535
6,36,NY,New York,01779796
7,37,NC,North Carolina,01027616
8,38,ND,North Dakota,01779797
9,39,OH,Ohio,01085497


# Get Topographic from Earth Engine assets
[Global ALOS mTPI (Multi-Scale Topographic Position Index)](https://developers.google.com/earth-engine/datasets/catalog/CSP_ERGo_1_0_Global_ALOS_mTPI#description)
The image layer is made up of many polygons with a "band" value. The lat/long point will fall within a polygon, and the value of band (in this case relative elevation) will be returned.

  <p>The mTPI distinguishes ridge from valley forms. It is calculated using
elevation data for each location subtracted by the mean elevation within a
neighborhood. mTPI uses moving windows of radius (km): 115.8, 89.9, 35.5,
13.1, 5.6, 2.8, and 1.2. It is based on the 30m "AVE" band of JAXA's ALOS
DEM (available in EE as JAXA/ALOS/AW3D30_V1_1).</p>

```
**Resolution**
270 meters

**Bands**
Name	    Units	  Min	    Max	
AVE	      Meters	-3758*	10963*	
Description: ALOS-derived mTPI ranging from negative (valleys) to positive (ridges) values
```

* estimated min or max value
<p>The Conservation Science Partners (CSP) Ecologically Relevant Geomorphology
(ERGo) Datasets, Landforms and Physiography contain detailed, multi-scale
data on landforms and physiographic (aka land facet) patterns. Although
there are many potential uses of these data, the original purpose for these
data was to develop an ecologically relevant classification and map of
landforms and physiographic classes that are suitable for climate adaptation
planning. </p>

In [ ]:
from time import sleep
import pandas as pd

# Check a known point
# Point (-75.7778, 40.4375) at 76m/px
# Pixels
# Lithology: Image (1 band)
# b1: 3
# ALOS mTPI: Image (1 band)
# AVE: -5

dem = ee.Image('CSP/ERGo/1_0/Global/ALOS_mTPI')

def get_band_value(long: float, lat: float):
  xy = ee.Geometry.Point([long,lat])
  data = dem.sample(xy, 100, dropNulls=True).first().get('AVE').getInfo()
  #print(counter, 'Litholoy band value', lat, long, data )
  return data

'''0 site_no        350160106324201
dec_lat_va           35.033358
dec_long_va        -106.545522
'''
print(datetime.datetime.now(), 'START')

for j, row in df_state_cds.iterrows():
  state_postal_abbreviation = row["state_postal_abbreviation"]
  print("state_cd:", state_postal_abbreviation)
  df_filtered = df[df['state_cd'] == row["state_fips_code"]]
  filename = f'mtpi_{state_postal_abbreviation}.csv'
  file = open(filename,'w')
  for i, row in df_filtered.iterrows():
    try:
      val = get_band_value(row["dec_long_va"], row["dec_lat_va"])
      file.writelines(str(i) + "," + row["site_no"] + "," + str(val) + '\n')
    except BaseException as err:
      print(f"   Unexpected {err}, {type(err)}")
      print('ERROR processed:', i, row.to_json(), val, " "*10, datetime.datetime.now())
      continue

    if (i%1000==0): 
      print('processed:',i,row["site_no"], val,state_postal_abbreviation ," "*10, datetime.datetime.now())
      sleep(30) # used to prevent the quota warning
  file.close()
  upload_blob('msd8654-498-dev-usgs', filename, filename)
print(datetime.datetime.now(), 'END')


2022-05-28 17:27:09.651805 START
state_cd: MT
processed: 313000 453801112411201 -3 MT            2022-05-28 17:30:19.644805
processed: 315000 452157107223801 -5 MT            2022-05-28 17:31:37.454150
processed: 316000 482203105323601 1 MT            2022-05-28 17:34:33.913911
processed: 323000 464025104572901 -4 MT            2022-05-28 17:36:13.799918
processed: 324000 483116104063401 3 MT            2022-05-28 17:39:43.604491
processed: 332000 474920104241501 -5 MT            2022-05-28 17:43:35.904265
processed: 333000 482317107493601 0 MT            2022-05-28 17:45:48.337401
processed: 336000 453456111092702 -6 MT            2022-05-28 17:49:19.707464
processed: 337000 463320110053301 -1 MT            2022-05-28 17:52:20.508652
processed: 338000 470143114193401 -15 MT            2022-05-28 17:55:36.671488
processed: 339000 480722114113501 0 MT            2022-05-28 17:58:39.750675
processed: 340000 481839114175802 0 MT            2022-05-28 18:01:37.341925
processed: 341000 4736

## Testing section for single point

In [ ]:
#ERROR processed: 26637 {"site_no":"353545121072901","station_nm":"027S008E08R005M","site_tp_cd":"GW","lat_va":"353545","long_va":"1210729",
# "dec_lat_va":35.5958041,"dec_long_va":-121.1257494,"district_cd":"6","state_cd":"06","county_cd":"79","well_depth_va":145.0,"well_depth_hue":4} 
#None            2022-05-21 04:19:59.902246
# processed: 27000 335734118165601 None CA            2022-05-21 04:25:19.321003
#   Unexpected Element.get: Parameter 'object' is required., <class 'ee.ee_exception.EEException'>

dem = ee.Image('CSP/ERGo/1_0/Global/ALOS_mTPI')

def get_lith(long: float, lat: float):
  xy = ee.Geometry.Point([long,lat])
  data = dem.sample(xy, 100, dropNulls=True).first().get('AVE').getInfo()
  #print(counter, 'Litholoy band value', lat, long, data )
  return data

val = get_lith(-121.1257494, 35.5958041)
print(val)

-8
